In [1]:
import pandas as pd
import numpy as np
import json
from shapely.geometry import shape, Point
from multiprocessing import  Pool
import html
import warnings
warnings.filterwarnings('ignore')

## Limpiar los datos

En ese workshop vamos utilizar un dataset del [National UFO Reporting Center (NUFORC)](http://www.nuforc.org/index.html), una organización norte americana para registro e investigación de avistamientos de OVNIs.

Los datos están en `../data/scrubbed.csv`, le vamos a limpiar un poco usando [`pandas`](https://pandas.pydata.org/) para poder crear nuestro dashboard de visualización usando [`streamlit`](https://streamlit.io/).

## Cargando e investigando datos

In [2]:
df = pd.read_csv("../data/scrubbed.csv", low_memory=False)

In [3]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,report_link
0,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,"Green/Orange circular disc over Chester, Engla...",1/21/2008,53.2,-2.916667,http://www.nuforc.org/webreports/060/S60217.html
1,10/10/1961 19:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.5950000,-82.188889,http://www.nuforc.org/webreports/055/S55782.html
2,10/10/1966 20:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111,http://www.nuforc.org/webreports/068/S68593.html
3,10/10/1968 19:00,brevard,nc,us,fireball,180,3 minutes,silent red/orange mass of energy floated by th...,6/12/2008,35.2333333,-82.734444,http://www.nuforc.org/webreports/062/S62666.html
4,10/10/1970 19:00,manchester,ky,us,unknown,180,3 minutes,"Slow moving, silent craft accelerated at an un...",2/14/2008,37.1536111,-83.761944,http://www.nuforc.org/webreports/061/S61389.html


In [4]:
df.shape

(53827, 12)

In [5]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude ', 'report_link'],
      dtype='object')

Quitamos el espacio al final del nombre de la columna `longitude ` para evitar problemas.

In [6]:
df.rename(columns={"longitude ":"longitude"}, inplace=True)

In [7]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
report_link              object
dtype: object

Utilizamos `regex` para remover caracteres no numericos de la columna latitude y convertirla al tipo float. Hacemos lo mismo con los segundos.

In [8]:
df.latitude = df.latitude.str.replace(r"[^0-9\.-]","")
df.latitude = df.latitude.astype(float)
df["duration (seconds)"] = df["duration (seconds)"].str.replace(r"[^0-9\.]","")
df["duration (seconds)"] = df["duration (seconds)"].astype(float)

In [9]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)      float64
duration (hours/min)     object
comments                 object
date posted              object
latitude                float64
longitude               float64
report_link              object
dtype: object

Empezamos a tratar los nulos, a empezar por los países.

In [10]:
df.isna().sum()

datetime                   0
city                       0
state                   3403
country                 5694
shape                    719
duration (seconds)         0
duration (hours/min)       0
comments                  14
date posted                0
latitude                   0
longitude                  0
report_link                0
dtype: int64

In [11]:
df.country.unique()

array(['gb', 'us', nan, 'ca', 'au', 'de'], dtype=object)

Utilizaremos el formato Alpha 3 para los codigos de identificación de los países, empezando por los que ya tenemos.

In [12]:
codes = {'us':"USA", 'gb':"GBR", 'ca':"CAN", 'au':"AUS", 'de':"DEU"}
df.country = df.country.apply(lambda c: codes.get(c,c))

Utilizaremos los datos geograficos del fichero `../data/countries.geojson`, que contiene las coordinadas de las fronteras de cada país para encontrar el país correspondiente. También crearemos un dicionário de los códigos y los nombres de los países para uso futuro.

In [13]:
with open('../data/countries.geojson') as f:
    countries_json = json.load(f)

countries = {}
code_name = {}
for feature in countries_json['features']:
    countries[feature["properties"]["ISO_A3"]] = shape(feature['geometry'])
    code_name[feature["properties"]["ISO_A3"]] = feature["properties"]["ADMIN"]
name_code = {v:k for k,v in code_name.items()}

with open('../data/code_name.json', 'w+') as outfile:
    json.dump(code_name, outfile)
    
with open('../data/name_code.json', 'w+') as outfile:
    json.dump(name_code, outfile)

In [14]:
def get_country(row,countries=countries):
    pt = Point(row["longitude"],row["latitude"])
    for code, geometry in countries.items():
        if geometry.contains(pt):
            return code
    return np.nan

Como el dataframe es bastante grande y la función `get_country` contiene una iteración que se puede hacer larga, utilizaremos del proceso de paralelización para aprovechar los diferentes nucleos del ordenador y hacerlo más rápido.

In [15]:
def parallelize_dataframe(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
def get_all_countries(df):
    df.country = df.apply(get_country,axis=1)
    return df

Buscaremos valores de país solo para aquellas filas que ya no contengan un codigo de 3 letras en la celda correspondiente.

In [16]:
missing_countries = df[~df.country.str.fullmatch("[A-Z]{3}",na=False)]

In [17]:
missing_countries.shape

(5694, 12)

In [18]:
df_co = parallelize_dataframe(missing_countries, get_all_countries)

Sustituímos los nuevos valores encontrados y removemos los que permaneceran nulos, i.e.: aquellos que no hemos conseguido encontrar el país correspondiente. Esos podrían ser datos con las coordinadas mal registradas o que ocurrieron sobre areas del oceano.

In [19]:
df["country"][df_co.index] = df_co.country

In [20]:
df = df[~df.country.isna()]

Aprovecharemos ahora de una misma función y el proceso de paralelización para hacer arreglos a los datos:
- Convertir a mayúsculas los nombres de los estados si el país es "USA" o por "--" si está en otro país.
- Removeremos información adicional del nombre de la ciudad, todo lo que esté contido entre paréntesis y también usaremos la primera letra en mayúscula.
- Convertiremos caracteres en codificación HTML a unicode.
- Haremos que la columna `shape` también esté con letras mayúsculas

In [21]:
def fix_data(df):
    df.state = df.apply(lambda row: str(row["state"]).upper() if row["country"]=="USA" else "--",
                           axis=1)
    df.city = df.city.str.replace(r"\(.*\)","").str.strip().str.title().apply(lambda x: html.unescape(x))
    df.comments = df[~df.comments.isna()].comments.apply(lambda x: html.unescape(x))
    df["shape"] = df["shape"].str.title()
    df['datetime'] = df['datetime'].str.replace('24:00', '0:00')
    return df

In [22]:
df = parallelize_dataframe(df, fix_data)

Sustituimos valores que permanecen nulos por etiquetas de desconocido y exportamos el csv limpio.

In [23]:
df["shape"][df["shape"].isna()] = "Unknown"
df["shape"][df["shape"] == "Changed"] = "Changing"
df["comments"][df["comments"].isna()] = "No Comment"

In [24]:
df.isna().sum()

datetime                0
city                    0
state                   0
country                 0
shape                   0
duration (seconds)      0
duration (hours/min)    0
comments                0
date posted             0
latitude                0
longitude               0
report_link             0
dtype: int64

Separating year, month, hour and weekday from datetime.

In [25]:
datetime = pd.DatetimeIndex(df.datetime)
df["year"] = datetime.year
df["month"] = datetime.month
df["hour"] = datetime.hour
df["weekday"] = datetime.weekday

In [26]:
df.to_csv("../data/clean.csv", index=False)

In [27]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,report_link,year,month,hour,weekday
0,10/10/1955 17:00,Chester,--,GBR,Circle,20.0,20 seconds,"Green/Orange circular disc over Chester, Engla...",1/21/2008,53.200000,-2.916667,http://www.nuforc.org/webreports/060/S60217.html,1955,10,17,0
1,10/10/1961 19:00,Bristol,TN,USA,Sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,http://www.nuforc.org/webreports/055/S55782.html,1961,10,19,1
2,10/10/1966 20:00,Pell City,AL,USA,Disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111,http://www.nuforc.org/webreports/068/S68593.html,1966,10,20,0
3,10/10/1968 19:00,Brevard,NC,USA,Fireball,180.0,3 minutes,silent red/orange mass of energy floated by th...,6/12/2008,35.233333,-82.734444,http://www.nuforc.org/webreports/062/S62666.html,1968,10,19,3
4,10/10/1970 19:00,Manchester,KY,USA,Unknown,180.0,3 minutes,"Slow moving, silent craft accelerated at an un...",2/14/2008,37.153611,-83.761944,http://www.nuforc.org/webreports/061/S61389.html,1970,10,19,5
